In [2]:
import pandas as pd


meta_df = pd.read_csv("/content/channel details.csv")
transcript_df = pd.read_csv("/content/video_transcripts.csv")

print("📊 Metadata Dataset")
print(meta_df.info())
print(meta_df.describe(include='all'))
print("\nMissing values per column:")
print(meta_df.isnull().sum())
print("\nDuplicates:", meta_df.duplicated().sum())

print("\n📊 Transcript Dataset")
print(transcript_df.info())
print(transcript_df.describe(include='all'))
print("\nMissing values per column:")
print(transcript_df.isnull().sum())
print("\nDuplicates:", transcript_df.duplicated().sum())

📊 Metadata Dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   videoId               50 non-null     object
 1   title                 50 non-null     object
 2   description           50 non-null     object
 3   publishedAt           50 non-null     object
 4   tags                  50 non-null     object
 5   categoryId            50 non-null     int64 
 6   defaultLanguage       50 non-null     object
 7   defaultAudioLanguage  50 non-null     object
 8   thumbnails            50 non-null     object
 9   duration              50 non-null     object
 10  viewCount             50 non-null     int64 
 11  likeCount             50 non-null     int64 
 12  commentCount          50 non-null     int64 
 13  privacyStatus         50 non-null     object
 14  channel_id            50 non-null     object
 15  channel_title         5

cleaning the dataset


In [4]:
import pandas as pd


meta_df = pd.read_csv("/content/channel details.csv")
transcript_df = pd.read_csv("/content/video_transcripts.csv")

print("Before Cleaning:")
print("Metadata shape:", meta_df.shape)
print("Transcript shape:", transcript_df.shape)


meta_df['publishedAt'] = pd.to_datetime(meta_df['publishedAt'], errors='coerce')

meta_df = meta_df.drop_duplicates(subset=["title"], keep="first")


meta_df['description'] = meta_df['description'].fillna("")

print("\nAfter Metadata Cleaning:")
print("Shape:", meta_df.shape)
print("Missing values per column:\n", meta_df.isnull().sum())


#  Clean Transcript Dataset

if "transcript" in transcript_df.columns:
    transcript_df = transcript_df[["video_id", "transcript"]]
else:
    transcript_df = transcript_df.rename(columns={"id": "video_id"})[["video_id", "transcript"]]


transcript_df = transcript_df.dropna(subset=["transcript"])


transcript_df = transcript_df.drop_duplicates(subset=["video_id"], keep="first")

print("\nAfter Transcript Cleaning:")
print("Shape:", transcript_df.shape)
print("Missing values per column:\n", transcript_df.isnull().sum())


meta_df.to_csv("channel details_cleaned.csv", index=False, encoding="utf-8")
transcript_df.to_csv("video transcripts_cleaned.csv", index=False, encoding="utf-8")

print("\n💾 Cleaned datasets saved as:")
print("channel details_cleaned.csv")
print("video transcripts_cleaned.csv")

Before Cleaning:
Metadata shape: (50, 21)
Transcript shape: (327, 12)

After Metadata Cleaning:
Shape: (50, 21)
Missing values per column:
 videoId                 0
title                   0
description             0
publishedAt             0
tags                    0
categoryId              0
defaultLanguage         0
defaultAudioLanguage    0
thumbnails              0
duration                0
viewCount               0
likeCount               0
commentCount            0
privacyStatus           0
channel_id              0
channel_title           0
channel_description     0
channel_country         0
channel_thumbnail       0
subscriberCount         0
videoCount              0
dtype: int64

After Transcript Cleaning:
Shape: (327, 2)
Missing values per column:
 video_id      0
transcript    0
dtype: int64

💾 Cleaned datasets saved as:
channel details_cleaned.csv
video transcripts_cleaned.csv
